In [3]:
from collections import defaultdict
from random import choice, randint, random
import time

In [4]:
operand_range = list(range(1, 11))

In [5]:
# class AnswerHistory(self):
#     def __init__(self):

In [6]:
class Player:
    def __init__(self):
        self.history = defaultdict(list)
    
    def update(self, question, answer):
        self.history[question].append(answer)
        
    def answer(self, question):
        if question in self.history:
            return choice(self.history[question])
        else:
            return Answer(choice(operand_range), 5*(1 + random()))
    
    def __repr__(self):
        return '\n'.join([f'{question}: {" ".join([str(answer) for answer in answers])}' for question, answers in self.history.items()])

In [7]:
class Answer:
    def __init__(self, answer, time_taken):
        self.answer = answer
        self.time_taken = time_taken
        
    def __repr__(self):
        return f'{self.answer} in {self.time_taken:0.3f}s'

In [22]:
class Question:
    def __init__(self, operation, first_operand, second_operand):
        self.operation = operation
        self.first_operand = first_operand
        self.second_operand = second_operand
    
    def __repr__(self):
        return ' '.join([
            str(self.first_operand),
            self.operation,
            str(self.second_operand)
        ])
    
    def __eq__(self, other):
        return self.__hash__() == hash(other)
    
    def get_answer(self):
        return eval(str(self))
    
    def is_attempt_correct(self, attempt):
        return self.get_answer() == attempt
    
    def __hash__(self):
        return hash((self.operation, *sorted((self.first_operand, self.second_operand))))

In [9]:
class QuestionGenerator:
    def __init__(self, operation, main_operand=None, min_operand=None, max_operand=None):
        self.operation = operation
        if main_operand:
            self.main_operand = main_operand
        else:
            self.operand_range = list(range(self.min_operand, max_operand + 1))
    
    def get(self):
        second_operand = choice(operand_range)
        if self.main_operand:
            return Question(self.operation, self.main_operand, second_operand)
        else:
            return Question(self.operation, choice(operand_range), second_operand)

In [10]:
class TrainingSession:
    def __init__(self, player, n_questions, operation, main_operand=None, min_operand=None, max_operand=None):
        self.n_questions = n_questions
        self.questionGenerator = QuestionGenerator(operation, main_operand, min_operand, max_operand)
        self.player = player
    
    def train(self):
        for i in range(self.n_questions):
            question = self.questionGenerator.get()
            t0 = time.time()
            answer = int(input(f'{question} = '))
            t1 = time.time()
            correct = question.is_attempt_correct(answer)
            time_taken = t1 - t0
            self.player.update(question, Answer(answer, time_taken))
            print('Correct' if correct else 'Wrong', f'in {time_taken:0.3f}s')

In [11]:
class Competition:
    def __init__(self, player1, player2, n_questions, operation, operand):
        self.player1 = player1
        self.player2 = player2
        self.n_questions = n_questions
        self.questionGenerator = QuestionGenerator(operation, operand)

    def compete(self):
        for i in range(self.n_questions):
            question = self.questionGenerator.get()
            print(question)
            player1_answer = self.player1.answer(question)
            player2_answer = self.player2.answer(question)
            player1_answer.answer
            
            print('Player 1:', a1, '| Player 2:', a2)

In [12]:
p1 = Player()
p2 = Player()

In [19]:
p1.history

defaultdict(list,
            {2 * 8: [16 in 3.026s],
             2 * 5: [10 in 2.040s],
             2 * 2: [4 in 1.214s, 4 in 1.120s]})

In [13]:
TrainingSession(p1, 4, '*', 2).train()

2 * 8 = 16
Correct in 3.026s
2 * 5 = 10
Correct in 2.040s
2 * 2 = 4
Correct in 1.214s
2 * 2 = 4
Correct in 1.120s


In [14]:
TrainingSession(p2, 4, '*', 2).train()

2 * 6 = 12
Correct in 1.206s
2 * 7 = 14
Correct in 1.685s
2 * 3 = 6
Correct in 1.371s
2 * 1 = 2
Correct in 0.943s


In [15]:
c = Competition(p1, p2, 4, '*', 2)

In [16]:
c.compete()

2 * 9
Player 1: 10 in 7.522s | Player 2: 2 in 8.614s
2 * 2
Player 1: 4 in 1.214s | Player 2: 3 in 8.850s
2 * 5
Player 1: 10 in 2.040s | Player 2: 1 in 7.161s
2 * 10
Player 1: 9 in 5.997s | Player 2: 7 in 9.019s
